In [ ]:
import sys
import random
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import plot_precision_recall_curve
from sklearn.linear_model import LogisticRegression

In [ ]:
##################
# You can modify file names and path to files as you need.
path_to_data_file = 'data_file.txt'
performance_file = 'logistic_hyperparameter_tuning.txt'
##################

iteration = int(sys.argv[1])

whole_data = [line.strip().split('\t') for line in open(path_to_data_file)]

del(whole_data[0])

featDic = {}
data_list_x = []
data_list_y = []

for line in whole_data:
    featDic[line[0]] = list(map(float,line[1:11])) + list(map(int,line[11:13])) + list(map(float,line[13:15])) + list(map(int,line[15]))
    data_list_x.append(featDic[line[0]][:-1])
    data_list_y.append(featDic[line[0]][-1])
    
data_x = np.array(data_list_x)
data_y = np.array(data_list_y)

In [ ]:
seed_1 = 1209 # random number
np.random.seed(seed_1)
n_seed = np.random.randint(10000)

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=n_seed)

accs = []
ROC_aucs = []
f1s = []
RP_aucs = []

for train, test in cv.split(data_x, data_y):
    
    clf = LogisticRegression(random_state=123).fit(data_x[train], data_y[train])

    test_acc = clf.score(data_x[test], data_y[test])
    
    predictions = clf.predict_proba(data_x[test])[:,1]
    
    fpr, tpr, threshold = metrics.roc_curve(data_y[test], predictions)
    roc_auc = metrics.auc(fpr, tpr)
    
    accs.append(test_acc)
    ROC_aucs.append(roc_auc)
    
    precision, recall, thresholds = precision_recall_curve(data_y[test], predictions)
    f1 = f1_score(data_y[test], predictions.round())
    f1s.append(f1)
        
    rp_auc = metrics.auc(recall, precision)
    RP_aucs.append(rp_auc)
    
mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = metrics.auc(mean_fpr, mean_tpr)

In [ ]:
with open(performance_file, 'a') as perf_file:
    perf_file.write('\t'.join([str(n_seed), str(iteration), str(sum(accs) / 3), '\t'.join(str(x) for x in accs), 
                               str(mean_auc), '\t'.join(str(x) for x in ROC_aucs),
                               str(sum(f1s)/3), '\t'.join(str(x) for x in f1s),
                               str(sum(RP_aucs) / 3), '\t'.join(str(x) for x in RP_aucs)]) + '\n')